In [1]:
# Notebook tests for tar aliases (e.g., sort-tar-archive, tar-this-dir-dated, etc)

# Note: The aliases should be defined before Jupyter is invoked.
# For BatsPP, they can loaded via '--source all-tomohara-aliases-etc.bash' option.

In [2]:
#Global Setup

TMP=/tmp/temp-aliases-test

mkdir $TMP --parents
command cd $TMP

In [3]:
# extract-tar tests
# create tar to later extract
mkdir tarFolder
command cd tarFolder
echo "tar this" > tar.txt
command cd $TMP
tar -zcvf tared.tar.gz tarFolder && rm -rf tarFolder

/tmp/temp-aliases-test/tarFolder
tarFolder/
tarFolder/tar.txt
removed 'tarFolder/tar.txt'
removed directory 'tarFolder'


In [4]:
# extract the tar just created
extract-tar tared.tar.gz

tarFolder/
tarFolder/tar.txt


In [5]:
# Assert the files are extracted
[ $(ls | grep Folder | wc -l) -eq 1 ]; echo $?;

0


In [6]:
#extract-tar-force tests
echo "tar this 2" >> tarFolder/tar.txt
prev_lines=$(cat tarFolder/tar.txt | wc -l)
extract-tar-force tared.tar.gz

tarFolder/
tarFolder/tar.txt


In [7]:
#Assert amount of lines after and before extract-force
[ $prev_lines -eq 2 ]; echo $?;
[ $(cat tarFolder/tar.txt | wc -l) -eq 1 ]; echo $?;

0
0


In [8]:
# un-tar tests
# same as extract-tar
rm -rf tarFolder
un-tar tared.tar.gz

removed 'tarFolder/tar.txt'
removed directory 'tarFolder'
tarFolder/
tarFolder/tar.txt


In [9]:
# Assert the files are extracted
[ $(ls | grep tarFolder | wc -l) -eq 1 ]; echo $?;

0


In [10]:
#extract-tar-here tests
rm -rf tarFolder
mkdir testFolder
command cd $TMP
command cd testFolder
extract-tar-here tared.tar.gz

removed 'tarFolder/tar.txt'
removed directory 'tarFolder'
/tmp/temp-aliases-test/testFolder
/tmp/temp-aliases-test /tmp/temp-aliases-test/testFolder
]1;$ temp-aliases-test [/tmp/temp-aliases-test]]2;$ temp-aliases-test [/tmp/temp-aliases-test]tarFolder/
tarFolder/tar.txt
/tmp/temp-aliases-test/testFolder
]1;$ testFolder [/tmp/temp-aliases-test/testFolder]]2;$ testFolder [/tmp/temp-aliases-test/testFolder]


In [11]:
# Assert the files are extracted
command cd $TMP
[ $(ls | grep tarFolder | wc -l) -eq 1 ]; echo $?;

0


In [12]:
# view-tar tests
# Assert the existence of two dirs inside tar
[ $(view-tar tared.tar.gz | grep tarFolder | wc -l) -eq 2 ]; echo $?;

drwxrwxr-x: command not found
1


In [13]:
command cd $HOME
rm -rf $TMP

removed '/tmp/temp-aliases-test/tarFolder/tar.txt'
removed directory '/tmp/temp-aliases-test/tarFolder'
removed '/tmp/temp-aliases-test/tared.tar.gz'
removed directory '/tmp/temp-aliases-test/testFolder'
removed directory '/tmp/temp-aliases-test'
